# Proyecto ecuacion de calor en Python
Utilizando el metodo de diferencias finitas para resolver la ecuacion en dos dimensiones espaciales.

In [25]:
import numpy as np

def fcts_calor(Ti, Td, Tar, Tab, Tp, kappa, L, a, M, N, h, t_fin):
    T = np.zeros((M+1, N+1), dtype = float)
    #condiciones iniciales
    T[0:M+1, 0] = Ti
    T[0:M+1, N] = Td
    T[0, 0:N+1] = Tar
    T[M, 0:N+1] = Tab
    T[1:M, 1:N] = Tp
    
    #arreglo para hacer las operaciones con FCTS
    Tk = np.zeros((M+1, N+1), dtype = float)
    Tk[0:M+1, 0] = Ti
    Tk[0:M+1, N] = Td
    Tk[0, 0:N+1] = Tar
    Tk[M, 0:N+1] = Tab

    #Los otros parametros
    t = 0.0
    dy= a/N  #Particiones del ancho
    dx = L/M  #Partciones del largo
    d1 = dy/dx  #dy/dx que es parte de la formula
    d2 = dx/dy  #dx/dy que es parte de la formula
    ites = 0
    indi = 0
    Fo = (kappa*t_fin)/(dy*dx)

    Tk = np.copy(T)

    #Aqui se implementa el FCTS
    while t < t_fin:
        for j in range(1,N):
            for i in range(1,M):
                value = Fo*(d1*(T[i-1,j] + T[i+1,j] - 2*T[i,j]) + d2*(T[i,j+1] + T[i,j-1] - 2*T[i,j])) + T[i,j]
                Tk[i,j] = value

        T = Tk
        t += h
    return T
        

In [26]:
valores = fcts_calor(100, 0, 0, 120, 10, 22.8, 10, 8, 100, 80, 1e-4, 180)

/tmp/ipykernel_7991/966880228.py:35: RuntimeWarning: overflow encountered in scalar multiply
  value = Fo*(d1*(T[i-1,j] + T[i+1,j] - 2*T[i,j]) + d2*(T[i,j+1] + T[i,j-1] - 2*T[i,j])) + T[i,j]
/tmp/ipykernel_7991/966880228.py:35: RuntimeWarning: overflow encountered in scalar add
  value = Fo*(d1*(T[i-1,j] + T[i+1,j] - 2*T[i,j]) + d2*(T[i,j+1] + T[i,j-1] - 2*T[i,j])) + T[i,j]
/tmp/ipykernel_7991/966880228.py:35: RuntimeWarning: invalid value encountered in scalar subtract
  value = Fo*(d1*(T[i-1,j] + T[i+1,j] - 2*T[i,j]) + d2*(T[i,j+1] + T[i,j-1] - 2*T[i,j])) + T[i,j]


KeyboardInterrupt: 

In [20]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

plt.imshow(valores)
plt.purple()
plt.show()

NameError: name 'valores' is not defined